In [1]:
import pandas as pd
import numpy as np
import os

from sklearn import ensemble, linear_model
from utils import *

In [7]:
df = pd.read_csv('./data/train.csv')
df_tst = pd.read_csv('./data/test.csv')
subm = pd.read_csv('./data/sample_submission.csv')

In [8]:
Ys = ['Y{}'.format(str(i).zfill(2)) for i in range(15, 17)]

In [9]:
df['Y'] = df.loc[:, Ys].mean(axis=1)

In [10]:
df['pre_Y'] = np.concatenate([[0], df['Y'].values[:-1]])

In [11]:
df = df[1:]

In [12]:
df_trn = df[df['Y18'].isna()]
df_trn['Y'] = df_trn.loc[:, Ys].mean(axis=1)
df_val = df[df['Y00'].isna()]

C:\Users\dhzns\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
import lightgbm as lgb

In [15]:
Xs = np.concatenate([X1, X2, X3, X4, X5, X6, X7, X8])

In [16]:
dm_trn = lgb.Dataset(df_trn[Xs], label=df_trn['Y'])

In [17]:
dm_val = lgb.Dataset(df_val[Xs], label=df_val['Y18'], reference=dm_trn)

In [18]:
param = {'num_leaves': 31, 'objective': 'regression'}
param['metric'] = 'mse'

In [19]:
num_round = 50
bst = lgb.train(param, dm_trn, num_round, valid_sets=[dm_val])

[1]	valid_0's l2: 49.7385
[2]	valid_0's l2: 42.8033
[3]	valid_0's l2: 36.542
[4]	valid_0's l2: 31.4966
[5]	valid_0's l2: 27.178
[6]	valid_0's l2: 23.7245
[7]	valid_0's l2: 21.1287
[8]	valid_0's l2: 19.0419
[9]	valid_0's l2: 17.1766
[10]	valid_0's l2: 15.4854
[11]	valid_0's l2: 14.3831
[12]	valid_0's l2: 13.2965
[13]	valid_0's l2: 12.5303
[14]	valid_0's l2: 11.8776
[15]	valid_0's l2: 11.2805
[16]	valid_0's l2: 10.7763
[17]	valid_0's l2: 10.3569
[18]	valid_0's l2: 10.0065
[19]	valid_0's l2: 9.63115
[20]	valid_0's l2: 9.36652
[21]	valid_0's l2: 9.16007
[22]	valid_0's l2: 8.97219
[23]	valid_0's l2: 8.89233
[24]	valid_0's l2: 8.72318
[25]	valid_0's l2: 8.57317
[26]	valid_0's l2: 8.47758
[27]	valid_0's l2: 8.42179
[28]	valid_0's l2: 8.36391
[29]	valid_0's l2: 8.3094
[30]	valid_0's l2: 8.26925
[31]	valid_0's l2: 8.1867
[32]	valid_0's l2: 8.14525
[33]	valid_0's l2: 8.09839
[34]	valid_0's l2: 8.04816
[35]	valid_0's l2: 7.99293
[36]	valid_0's l2: 7.98122
[37]	valid_0's l2: 7.96841
[38]	valid_0's

In [20]:
mdl = lgb.LGBMRegressor()

In [21]:
mdl = ensemble.RandomForestRegressor(n_estimators=100)

In [22]:
# mdl = linear_model.Lasso(alpha=0.8)

mdl.fit(df_trn[Xs], df_trn['Y'])
pred = mdl.predict(df_val[Xs])

pred_adj = pred + 0
print(mse_AIFrenz(df_val['Y18'], pred_adj))

7.806127647569444


# iter

In [107]:
preds_val = []
pre_y_init = mdl.predict([df_trn[Xs].iloc[-1]])

pre_y = pre_y_init
for i in range(len(df_val)):
    print('\r {}/{}'.format(str(i).zfill(4), len(df_val)), end='')
    datum_val = np.concatenate([df_val[Xs[:-1]].iloc[i], pre_y])
    pred = mdl.predict([datum_val])
    preds_val.append(pred[0])
    pre_y = pred

mse_AIFrenz(df_val['Y18'], preds_val)

 0431/432

13.741428058501482

In [93]:
preds = []
pre_y_init = mdl.predict([df_val[Xs].iloc[-1]])

pre_y = pre_y_init
for i in range(len(df_tst)):
    print('\r {}/{}'.format(str(i).zfill(4), len(df_tst)), end='')
    datum_tst = np.concatenate([df_tst[Xs[:-1]].iloc[i], pre_y])
    pred = mdl.predict([datum_tst])
    preds.append(pred[0])
    pre_y = pred

 11519/11520

In [90]:
preds

[array([20.9718111]),
 array([20.85494571]),
 array([20.72447903]),
 array([20.58161726]),
 array([20.44700395]),
 array([20.39522348]),
 array([20.28671559]),
 array([20.19224628]),
 array([20.17154545]),
 array([20.15432792]),
 array([20.07405826]),
 array([19.96336722]),
 array([19.91349786]),
 array([19.84709213]),
 array([19.73796631]),
 array([19.641637]),
 array([19.60446838]),
 array([19.62133299]),
 array([19.6587075]),
 array([19.61942604]),
 array([19.59148176]),
 array([19.63761062]),
 array([19.69217742]),
 array([19.72124851]),
 array([19.66980351]),
 array([19.55433295]),
 array([19.48732226]),
 array([19.36977975]),
 array([19.44899891]),
 array([19.3916925]),
 array([19.31989124]),
 array([19.2510137]),
 array([19.18464178]),
 array([19.11998512]),
 array([19.05757832]),
 array([19.02030719]),
 array([19.07699259]),
 array([19.08732831]),
 array([19.08607234]),
 array([19.10783474]),
 array([19.11688599]),
 array([19.11363286]),
 array([19.16409917]),
 array([19.270048

# Test

In [17]:
pred_test = mdl.predict(df_tst[Xs])
pred_test_adj = pred_test+0.55

In [20]:
writeSubm(pred_test_adj)

Submission file is written on ./subm/subm_0307020539.csv


# End